# Limpieza de Datos

In [88]:
import pandas as pd
import unicodedata

## Limpieza de la variable "Teléfono"

In [89]:
df = pd.read_csv("all_data.csv", dtype={"TELEFONO": str})

In [90]:
# Cuántos valores únicos y cuántos vacíos
print(df["TELEFONO"].nunique(), "valores únicos")
print(df["TELEFONO"].isna().sum(), "NaN / vacíos")
print("Total rows:", df.shape[0])

4215 valores únicos
46 NaN / vacíos
Total rows: 6599


In [91]:
# Valores con notación científica (“.0”)  
mask_sci = df["TELEFONO"].str.contains(r"\.0$", na=False)
print("Con .0 al final:", mask_sci.sum())

# Valores con caracteres no numéricos  
mask_nonnumeric = df["TELEFONO"].str.contains(r"\D", na=False)
print("Con otros símbolos:", mask_nonnumeric.sum())

# Longitudes  
longitudes = df["TELEFONO"].dropna().str.replace(r"\D", "", regex=True).str.len()
print(longitudes.describe())


Con .0 al final: 3522
Con otros símbolos: 3556
count    6553.000000
mean        8.577598
std         0.782789
min         2.000000
25%         8.000000
50%         9.000000
75%         9.000000
max        24.000000
Name: TELEFONO, dtype: float64


In [92]:
import re

def limpiar_telefono(x):
    def clean_number(num_str):
        # 1) manejar notación científica
        try:
            if re.search(r"[eE]", num_str):
                num_str = str(int(float(num_str)))
        except:
            return "-1"
        # 2) quitar todo menos dígitos
        num_str = re.sub(r"\D+", "", num_str)
        if not num_str:
            return "-1"
        # 3) truncar si hay >8 dígitos
        if len(num_str) > 8:
            num_str = num_str[:8]
        # 4) invalidar si <7 dígitos
        if len(num_str) < 7:
            return "-1"
        # 5) asegurar 8 dígitos
        return num_str.zfill(8)

    if pd.isna(x):
        return pd.Series({
            "TELEFONO_LIMPIO": "-1",
            "TELEFONOS_ADICIONALES": ""
        })

    s = str(x).strip()
    # dividir en múltiplos separadores
    partes = re.split(r"[;\-/, ]+", s)

    # limpiar cada parte
    limpiados = [clean_number(p) for p in partes]
    # quedarnos sólo con los válidos
    validos = [n for n in limpiados if n != "-1"]

    if not validos:
        return pd.Series({
            "TELEFONO_LIMPIO": "-1",
            "TELEFONOS_ADICIONALES": ""
        })

    primary = validos[0]
    additional = validos[1:]

    return pd.Series({
        "TELEFONO_LIMPIO": primary,
        "TELEFONOS_ADICIONALES": ";".join(additional)
    })

In [93]:
# Aplicar la función y unir las dos columnas nuevas
df[["TELEFONO_LIMPIO", "TELEFONOS_ADICIONALES"]] = (
    df["TELEFONO"]
      .apply(limpiar_telefono)
)

In [94]:
# Ahora imprimimos las métricas sobre la columna limpia
print((df["TELEFONO_LIMPIO"] == "-1").sum(), "teléfonos inválidos")

51 teléfonos inválidos


In [95]:
print(df[['TELEFONO']].sample(5))

        TELEFONO
1019         NaN
2788    57543661
1664    59340090
4491  42245597.0
1126    23271111


In [96]:
print(df["TELEFONOS_ADICIONALES"])

0        
1        
2        
3        
4        
       ..
6594     
6595     
6596     
6597     
6598     
Name: TELEFONOS_ADICIONALES, Length: 6599, dtype: object


In [97]:
long_post = df["TELEFONO_LIMPIO"]\
    .replace("-1", pd.NA)\
    .dropna()\
    .str.len()\
    .value_counts()
print("Distribución de longitudes tras limpieza:\n", long_post)

Distribución de longitudes tras limpieza:
 TELEFONO_LIMPIO
8    6548
Name: count, dtype: int64


### Razones de las decisiones tomadas para la limpieza:

<small>

Decidimos primero normalizar todos los valores a cadenas de dígitos puros para garantizar la comparabilidad y evitar formatos mixtos (espacios, guiones, puntos o notación científica). Al eliminar cualquier carácter no numérico y convertir expresiones en “e” o con decimales a enteros, garantizamos que cada teléfono represente únicamente los dígitos esenciales. Truncar los números a ocho dígitos (o completar con ceros a la izquierda cuando falten) responde a la longitud estándar de los celulares en nuestro contexto, mientras que invalidar aquellas cadenas con menos de siete dígitos evita datos claramente erróneos o incompletos. Además, al extraer el primer número como “principal” y agrupar los adicionales en una columna separada, permitimos tanto el análisis prioritario de un contacto principal como la retención de otras vías de comunicación, sin duplicar registros.

</small>

## Limpieza de la variable "Director"

In [98]:
df = pd.read_csv("all_data.csv", dtype={"DIRECTOR": str})

In [99]:
def normalize_text(text: str) -> str:
    """
    Elimina tildes y acentos, pasa a mayúsculas y recorta espacios.
    """
    text = unicodedata.normalize('NFKD', text)
    text = ''.join(c for c in text if not unicodedata.combining(c))
    return text.upper().strip()

In [100]:
INVALID_MARKERS = {"nan", "none", "", "---", "----", "-----", "x", "--"}

In [101]:
def limpiar_director(val: str) -> str:
    """
    1) Si es nulo o un marcador inválido → "SIN DIRECTOR"
    2) Si, tras strip(), len < 6 → "SIN DIRECTOR"
    3) En otro caso → normalizar tildes y pasar a mayúsculas
    """
    if pd.isna(val):
        return "SIN DIRECTOR"
    v = val.strip()
    if v.lower() in INVALID_MARKERS:
        return "SIN DIRECTOR"
    if len(v) < 6:
        return "SIN DIRECTOR"
    return normalize_text(v)

In [102]:
df["DIRECTOR_LIMPIO"] = df["DIRECTOR"].apply(limpiar_director)

In [103]:
print("Total filas:                       ", len(df))
print("Nulos originales:                  ", df["DIRECTOR"].isna().sum())
print("Reemplazados como 'SIN DIRECTOR': ", (df["DIRECTOR_LIMPIO"] == "SIN DIRECTOR").sum())
print("Valores únicos en limpio:          ", df["DIRECTOR_LIMPIO"].nunique())
print("Top 10 directores más frecuentes:\n", df["DIRECTOR_LIMPIO"].value_counts().head(10))

Total filas:                        6599
Nulos originales:                   26
Reemplazados como 'SIN DIRECTOR':  40
Valores únicos en limpio:           3820
Top 10 directores más frecuentes:
 DIRECTOR_LIMPIO
SIN DIRECTOR                        40
MARIA DOLORES PEREZ TUCHAN          10
HECTOR REYNALDO GOMEZ AGUILAR        9
SONIA JOSEFINA MORALES CAXAJ         8
MELVIN RAFAEL REYES LOPEZ            8
RONI ERNESTO RECINOS ESTRADA         8
FRANCISCO REVOLORIO LOPEZ            8
SILVIA MARLENI ALVARADO GUARDADO     8
SANDRA NOEMI ORTIZ ESTRADA           7
EDILSAR ROCAEL VAZQUEZ PEREZ         7
Name: count, dtype: int64


In [ ]:
mapping = (
    df[["DIRECTOR", "DIRECTOR_LIMPIO"]]
      .dropna(subset=["DIRECTOR"])      
      .drop_duplicates()               
)

counts_raw_per_clean = (
    mapping
      .groupby("DIRECTOR_LIMPIO")["DIRECTOR"]
      .nunique()
)

variants = counts_raw_per_clean[counts_raw_per_clean > 1]

print("\nNombres normalizados que unifican >1 variante original:")
print(variants.sort_values(ascending=False).head(10))

print("\nEjemplos de variantes agrupadas:")
for clean_name, group in mapping.groupby("DIRECTOR_LIMPIO"):
    raws = list(group["DIRECTOR"])
    if len(raws) > 1:
        print(f"  {clean_name!r}  ←  {raws}")


Nombres normalizados que unifican >1 variante original:
DIRECTOR_LIMPIO
SIN DIRECTOR                              5
ANAHI DEL PILAR HERNANDEZ CHOCHOM         3
MARIA VERONICA AUYON OCAMPO DE GALICIA    3
BLANCA ROSA HERNANDEZ LOPEZ               2
ANGELICA DEL ROSARIO PATZAN QUISQUE       2
ANGEL FELIPE LARIOS MELENDEZ              2
CESAR FERNANDO ESCOBAR ZEPEDA             2
CARLA ELIZABETH MONTERROSO GOMEZ          2
CLAUDIA PATRICIA DAVILA RAMIREZ           2
DALIDA ANAYANZA HERNANDEZ RAMIREZ         2
Name: DIRECTOR, dtype: int64

Ejemplos de variantes agrupadas:
  'ANAHI DEL PILAR HERNANDEZ CHOCHOM'  ←  ['ANAHÍ DEL PILAR HERNÁNDEZ CHOCHOM', 'ANAHI DEL PILAR HERNÁNDEZ CHOCHÓM', 'ANAHÍ DEL PILAR HERNÁNDEZ CHOCHÓM']
  'ANGEL FELIPE LARIOS MELENDEZ'  ←  ['ANGEL FELIPE LARIOS MELENDEZ', 'ANGEL FELIPE LARIOS MELÉNDEZ']
  'ANGELICA DEL ROSARIO PATZAN QUISQUE'  ←  ['ANGÉLICA DEL ROSARIO PATZÁN QUISQUE', 'ANGELICA DEL ROSARIO PATZAN QUISQUE']
  'BLANCA ROSA HERNANDEZ LOPEZ'  ←  ['BLANCA 